In [19]:
import sys
sys.path.append('../../../utils')
sys.path.append('../../..')

from interpolate.markup_utils import load_markup, yolo_dataset_info
from src.metrics import compute_border_metrics, compute_precision_recall, compute_non_border_metrics

### Загрузка датасета и модели

In [20]:
CONFIG_PATH = '../../../config.json'
SPLIT = 'test'
IOU_THRESHOLD = 0.7

In [21]:
# Load config
import json
import numpy as np
from pathlib import Path

with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

# Load labels
dataset_info = yolo_dataset_info(Path(config['dense']))
gt_labels_dir = Path(dataset_info[SPLIT]) / 'labels'

In [22]:
MODEL_VERSION = 'no_background'

In [23]:
from ultralytics import YOLO
model = YOLO('/alpha/projects/wastie/code/kondrashov/tmp/dense_models/15_04_high_density_2cls.pt')

### Предсказание с лучшим по F1 confidence

In [24]:

import subprocess
import shutil
shutil.rmtree('runs/segment', ignore_errors=True)
# Run YOLO validation to get the best confidence score

# Run validation to get best confidence threshold
val_results = model.val(data=config['dense'], split=SPLIT)

best_f1_idx = np.argmax(val_results.seg.curves_results[1][1].mean(axis=0))
best_f1 = val_results.seg.curves_results[1][1][..., best_f1_idx].mean()
best_conf = val_results.seg.curves_results[1][0][best_f1_idx]
print(f"Best F1: {best_f1:.4f} at confidence {best_conf:.4f}")

# Create temporary directory for predictions
pred_labels_dir = Path('runs/segment/predict/labels')

# Run prediction with best confidence
model.predict(
    source=str(Path(dataset_info[SPLIT]) / 'images'),
    conf=best_conf,
    save_txt=True,
)


Ultralytics 8.3.0 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100 80GB PCIe, 81154MiB)
YOLOv8m-seg summary (fused): 263 layers, 24,586,614 parameters, 0 gradients, 98.7 GFLOPs


val: Scanning /alpha/projects/wastie/datasets/05_02_dense_test/test/labels.cache... 64 images, 2 backgrounds, 0 corrupt: 100%|██████████| 64/64 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.85s/it]


                   all         64       1300      0.878      0.799      0.889      0.737      0.877      0.798      0.884      0.685
                   bot         62       1173      0.895      0.803      0.907      0.742      0.893      0.801        0.9      0.692
                  alum         49        127       0.86      0.795      0.871      0.733       0.86      0.795      0.868      0.677
Speed: 6.7ms preprocess, 9.0ms inference, 0.0ms loss, 11.7ms postprocess per image
Results saved to runs/segment/val
Best F1: 0.8361 at confidence 0.3343

image 1/64 /alpha/projects/wastie/datasets/05_02_dense_test/test/images/tula_sep_0002_2024_07_16_14_17_15_000.jpg: 800x800 8 bots, 13.5ms
image 2/64 /alpha/projects/wastie/datasets/05_02_dense_test/test/images/tula_sep_0002_2024_07_16_14_17_18_000.jpg: 800x800 38 bots, 2 alums, 13.3ms
image 3/64 /alpha/projects/wastie/datasets/05_02_dense_test/test/images/tula_sep_0002_2024_07_16_14_17_21_000.jpg: 800x800 38 bots, 4 alums, 12.2ms
image 4/64 /

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'bot', 1: 'alum'}
 obb: None
 orig_img: array([[[ 2,  2,  2],
         [ 2,  2,  2],
         [ 2,  2,  2],
         ...,
         [18, 16, 16],
         [18, 16, 16],
         [18, 16, 16]],
 
        [[ 2,  2,  2],
         [ 2,  2,  2],
         [ 2,  2,  2],
         ...,
         [19, 17, 17],
         [19, 17, 17],
         [19, 17, 17]],
 
        [[ 2,  2,  2],
         [ 2,  2,  2],
         [ 2,  2,  2],
         ...,
         [20, 18, 18],
         [20, 18, 18],
         [20, 18, 18]],
 
        ...,
 
        [[ 6,  8,  5],
         [ 6,  8,  5],
         [ 6,  8,  5],
         ...,
         [ 5,  5,  5],
         [ 5,  5,  5],
         [ 6,  6,  6]],
 
        [[ 6,  8,  5],
         [ 6,  8,  5],
         [ 6,  8,  5],
         ...,
         [ 5,  5,  5],
         [ 5,  5,  5],
         [

### Подготавливаем данные

In [25]:
gt_paths = []
pred_paths = []
for gt_path in gt_labels_dir.glob("*.txt"):
    pred_path = pred_labels_dir / gt_path.name
    if not pred_path.exists():
        pred_path.touch()
    gt_paths.append(gt_path)
    pred_paths.append(pred_path)

### Считаем метрики

In [26]:
image_shape = (config['imgsz'], config['imgsz'])

In [27]:
both_metrics = compute_precision_recall(gt_paths, pred_paths, image_shape, IOU_THRESHOLD)
print(f"Metrics:\nPrecision: {both_metrics['precision']:.4f}\nRecall: {both_metrics['recall']:.4f}")

Masks processed: 100%|██████████| 8/8 [00:00<00:00, 643.89it/s]
Masks processed: 0it [00:00, ?it/s]
/alpha/projects/wastie/code/kondrashov/delta/notes/dense_models/per_mask/../../../utils/integrate/metrics.py:118: RuntimeWarning: Mean of empty slice.
  precision = pred_matched.mean()
/alpha/projects/wastie/code/kondrashov/delta/.venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/alpha/projects/wastie/code/kondrashov/delta/notes/dense_models/per_mask/../../../utils/integrate/metrics.py:119: RuntimeWarning: Mean of empty slice.
  recall = gt_matched.mean()
Masks processed: 100%|██████████| 25/25 [00:00<00:00, 186.35it/s]

Metrics:
Precision: 0.8489
Recall: 0.7908


In [28]:
border_metrics = compute_border_metrics(gt_paths, pred_paths, image_shape, IOU_THRESHOLD)
print(f"Border metrics:\nPrecision: {border_metrics['precision']:.4f}\nRecall: {border_metrics['recall']:.4f}")

Masks processed: 100%|██████████| 1/1 [00:00<00:00, 354.13it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 6/6 [00:00<00:00, 1675.38it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 5/5 [00:00<00:00, 1392.44it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 2/2 [00:00<00:00, 791.30it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 2/2 [00:00<00:00, 1465.52it/s]

Border metrics:
Precision: 0.7092
Recall: 0.8274


In [29]:
non_border_metrics = compute_non_border_metrics(gt_paths, pred_paths, image_shape, IOU_THRESHOLD)
print(f"Non border metrics:\nPrecision: {non_border_metrics['precision']:.4f}\nRecall: {non_border_metrics['recall']:.4f}")

Masks processed: 100%|██████████| 7/7 [00:00<00:00, 653.62it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 14/14 [00:00<00:00, 158.02it/s]
Masks processed: 0it [00:00, ?it/s]
Masks processed: 100%|██████████| 23/23 [00:00<00:00, 192.05it/s]

Non border metrics:
Precision: 0.8729
Recall: 0.7827


In [30]:
# Create a dictionary with all metrics for easy comparison
metrics_comparison = {
    'All objects': both_metrics,
    'Border objects': border_metrics,
    'Non-border objects': non_border_metrics
}

# Print comparison table
print("Metrics comparison:")
print("-" * 60)
print(f"{'Type':<20} {'Precision':>12} {'Recall':>12} {'F1-score':>12}")
print("-" * 60)

for metric_type, metrics in metrics_comparison.items():
    precision = metrics['precision']
    recall = metrics['recall']
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"{metric_type:<20} {precision:>12.4f} {recall:>12.4f} {f1:>12.4f}")

print("\nAnalysis:")
# Find best performing filter based on F1 score
best_f1 = 0
best_type = None

for metric_type, metrics in metrics_comparison.items():
    precision = metrics['precision']
    recall = metrics['recall']
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    if f1 > best_f1:
        best_f1 = f1
        best_type = metric_type

print(f"The best performing filter is '{best_type}' with F1-score of {best_f1:.4f}")

# Calculate F1 enhancement percentage
baseline = metrics_comparison['All objects']
best_opt = metrics_comparison[best_type]


baseline_f1 = 2 * (baseline['precision'] * baseline['recall']) / (baseline['precision'] + baseline['recall'])
best_f1_score = 2 * (best_opt['precision'] * best_opt['recall']) / (best_opt['precision'] + best_opt['recall'])

f1_enhancement = ((best_f1_score - baseline_f1) / baseline_f1) * 100
print(f"\nF1 score enhancement: {f1_enhancement:.2f}%")
print(f"Recall enhancement:    {(best_opt['recall'] - baseline['recall']) / baseline['recall'] * 100:.2f}%")
print(f"Precision enhancement: {(best_opt['precision'] - baseline['precision']) / baseline['precision'] * 100:.2f}%")
# Calculate error reduction coefficient
error_reduction = (1 - baseline_f1) / (1 - best_f1_score)
print(f"\nError reduction coefficient: {error_reduction:.3f}x ({(error_reduction - 1)*100:.1f}%)")

Metrics comparison:
------------------------------------------------------------
Type                    Precision       Recall     F1-score
------------------------------------------------------------
All objects                0.8489       0.7908       0.8188
Border objects             0.7092       0.8274       0.7637
Non-border objects         0.8729       0.7827       0.8253

Analysis:
The best performing filter is 'Non-border objects' with F1-score of 0.8253

F1 score enhancement: 0.80%
Recall enhancement:    -1.02%
Precision enhancement: 2.83%

Error reduction coefficient: 1.037x (3.7%)


In [31]:
print("The main result:")
f1_b = 2 * (metrics_comparison['Border objects']['precision'] * metrics_comparison['Border objects']['recall']) / (metrics_comparison['Border objects']['precision'] + metrics_comparison['Border objects']['recall'])
f1_n = 2 * (metrics_comparison['Non-border objects']['precision'] * metrics_comparison['Non-border objects']['recall']) / (metrics_comparison['Non-border objects']['precision'] + metrics_comparison['Non-border objects']['recall'])
main_error_coeff = (1-f1_b) / (1-f1_n)
print(f"\nError coefficient between Border and Non-Border masks: {main_error_coeff:.3f}x ({(main_error_coeff - 1)*100:.1f}%)")

The main result:

Error coefficient between Border and Non-Border masks: 1.353x (35.3%)


### Статистическая значимость результата

In [32]:
import numpy as np
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

def mcnemar_test(sample1, sample2, exact : bool = False):
    """
    Вычисляет критерий Макнимары для двух бинарных выборок.

    Параметры:
    ----------
    sample1 : list, numpy array или pandas Series
        Первая бинарная выборка (0 и 1).
    sample2 : list, numpy array или pandas Series
        Вторая бинарная выборка (0 и 1).
    exact : bool
        Флаг для использования хи-квадрат без аппроксимации. 
        Подходит для маленьких выборок. По умолчанию False.

    Возвращает:
    -----------
    stat : float
        Значение статистики критерия Макнимары.
    p_value : float
        p-value для проверки гипотезы.
    """
    # Проверка, что выборки имеют одинаковую длину
    if len(sample1) != len(sample2):
        raise ValueError("Выборки должны иметь одинаковую длину.")

    # Создание таблицы сопряженности 2x2
    table = pd.crosstab(sample1, sample2)

    # Проверка, что таблица 2x2
    if table.shape != (2, 2):
        raise ValueError("Таблица сопряженности должна быть 2x2.")

    # Вычисление критерия Макнимары
    result = mcnemar(table, exact=exact)
    stat = result.statistic
    p_value = result.pvalue

    return stat, p_value

##### Подготовим данные

In [33]:
b_conf = np.array(border_metrics['conf_matrix'], dtype=np.int32)
nb_conf = np.array(non_border_metrics['conf_matrix'], dtype=np.int32)
is_border = {}
is_matched = {}
for bc, nbc, title in [(b_conf, nb_conf, 'GT'), (b_conf.T, nb_conf.T, 'Pred')]:
    border_cnt = bc[:, 0].sum()
    non_border_cnt = nbc[:, 0].sum()
    part_is_border = [0] * non_border_cnt + [1] * border_cnt
    part_is_matched = [0] * nbc[1, 0] + [1] * nbc[0, 0]
    part_is_matched += [0] * bc[1, 0] + [1] * bc[0, 0]
    
    is_border[title] = part_is_border
    is_matched[title] = part_is_matched
    

##### 1. Проверим корреляцию между флагом, что объект краевой, и флагом, что объект верно предсказан 

In [34]:
total_is_border = is_border['GT'] + is_border['Pred']
total_is_matched = is_matched['GT'] + is_matched['Pred']
is_border['Total'] = total_is_border
is_matched['Total'] = total_is_matched

print("| Sample | Correlation coefficient | Sample size |")
print("|--------|-------------------------|-------------|")
print(f"| Total  | {np.corrcoef(total_is_border, total_is_matched)[0, 1]:>23.6f} | {len(total_is_border):>11} |")
print(f"| GT     | {np.corrcoef(is_border['GT'], is_matched['GT'])[0, 1]:>23.6f} | {len(is_border['GT']):>11} |")
print(f"| Pred   | {np.corrcoef(is_border['Pred'], is_matched['Pred'])[0, 1]:>23.6f} | {len(is_border['Pred']):>11} |")

| Sample | Correlation coefficient | Sample size |
|--------|-------------------------|-------------|
| Total  |               -0.056018 |        2511 |
| GT     |                0.036712 |        1300 |
| Pred   |               -0.167244 |        1211 |


##### 2. Посмотрим на корреляцию Спирмена, чтобы оценить статистическую значимость зависимости.

In [35]:
from scipy.stats import spearmanr

print("| Sample | Spearman correlation | p-value               |")
print("|--------|----------------------|-----------------------|")
for sample in ['GT', 'Pred', 'Total']:
    corr, p_value = spearmanr(is_border[sample], is_matched[sample])
    print(f"| {sample:>6} | {corr:>20.6f} | {p_value:>21} |")

| Sample | Spearman correlation | p-value               |
|--------|----------------------|-----------------------|
|     GT |             0.036712 |    0.1858829185212074 |
|   Pred |            -0.167244 | 4.761915677377471e-09 |
|  Total |            -0.056018 |  0.004986910391004618 |


Таким образом, на уровне значимости $\alpha=0.05$ 
1) зависимость крайне значима для предсказанных масок. Отвергаем гипотезу для Pred. 
2) незначима для GT масок, поэтому мы не можем отвергнуть гипотезу независимости GT.

##### 3. Применим Хи-квадрат для проверки независимости верного предсказания у краевых и некраевых объектов

In [36]:
from scipy.stats import chi2_contingency

for bc, nbc, title in [(b_conf, nb_conf, 'GT'), (b_conf.T, nb_conf.T, 'Pred')]:
    edge_objects = [0] * bc[1, 0] + [1] * bc[0, 0]
    non_edge_objects = [0] * nbc[1, 0] + [1] * nbc[0, 0]

    # Построение таблицы сопряженности
    # Строки: краевые и некраевые объекты
    # Столбцы: верное и неверное предсказание
    table = np.array([
        [sum(edge_objects), len(edge_objects) - sum(edge_objects)],  # Краевые объекты: верно, неверно
        [sum(non_edge_objects), len(non_edge_objects) - sum(non_edge_objects)]  # Некраевые объекты: верно, неверно
    ])

    # Применение критерия хи-квадрат
    chi2_stat, p_value, dof, expected = chi2_contingency(table)

    # Вывод p-value
    print(f"p-value критерия хи-квадрат ({title}): {p_value}")

p-value критерия хи-квадрат (GT): 0.22153314542790026
p-value критерия хи-квадрат (Pred): 1.1177774291308683e-08


Получили аналогичный результат

### Вывод

Между полученными метриками, корреляциями, критерием Спирмена и $\Chi^2$ нет противоречий.

- Критерии Спирмена и $\Chi^2$ сошлись во мнении о характере связи между краевым свойством и предсказанием масок для обоих групп.
- При не обнаруженной значимой связи между краевым свойством и предсказанием GT масок, разница recall для краевых и некраевых масок незначительна. Однако при значительной связи для Pred масок, мы видим худшую точность предсказания краевых масок, чем некраевых. Таким образом, результаты критериев согласуются с метриками.
- Корреляция между флагами, высокая для Pred и низкая для GT, согласуется с критериями. Согласно корреляции, существует обратная зависимость между флагом того, что предсказанный объект краевой, и флагом того, что объект предсказан верно. Это согласуется с метриками, где удаление краевых объектов приводит к улучшению точности.

Предлагается отмести краевые объекты как вносящие статистически значимый шум в предсказание.

В первом приближении, возможны следующие варианты:

1) Игнорировать вывод всех краевых объектов, предсказанных сетью.
2) Удалить краевые объекты из GT, тем самым мотивировав сеть отказаться от пресказания объектов на границах.